# Lab 9: Đối ngẫu và Phương pháp Newton – Bài thực hành đơn giản

Trong lab ngắn này, bạn sẽ ôn lại khái niệm đối ngẫu lồi qua một ví dụ nhỏ đủ tính toán được bằng tay và mã Python, sau đó triển khai phương pháp Newton với backtracking line search. Mục tiêu là hiểu công thức, tự viết mã, và trực quan hoá đường đi của thuật toán.


## Mục tiêu và tiền đề
Trong lab này, bạn sẽ: 
- (i) củng cố khái niệm đối ngẫu lồi qua một ví dụ nhỏ có thể tính được tay và bằng Python, 
- (ii) tự cài đặt thuật toán Newton với backtracking line search cho bài toán tối ưu hoá không ràng buộc và 
- (iii) trực quan hoá quỹ đạo hội tụ so với gradient descent để cảm nhận lợi ích của thông tin Hessian

## Thiết lập
Chạy ô sau để import và cấu hình cơ bản.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
np.random.seed(42)

from typing import Callable, Tuple


## Phần A — Ôn tập đối ngẫu qua ví dụ đơn giản
Xét bài toán primal lồi dạng bình phương tối ưu hoá có ràng buộc tuyến tính nhỏ:

$$
\min_{x\in\mathbb{R}^n} \; f(x)=\tfrac{1}{2}\lVert x\rVert^2 + c^\top x\quad \text{s.t.}\quad a^\top x = b.
$$

Hàm Lagrange: $$\mathcal{L}(x,\lambda) = \tfrac{1}{2}\lVert x\rVert^2 + c^\top x + \lambda(a^\top x - b)$$. Với $\lambda\in\mathbb{R}$, đối ngẫu là $g(\lambda)=\inf_x \mathcal{L}(x,\lambda)$. 

Ta tìm $x(\lambda)$ thoả $\nabla_x\mathcal{L}=0\Rightarrow x(\lambda)=-(c+\lambda a)$. Thay vào thu được hàm đối ngẫu lồi-concave theo $\lambda$, và bài toán dual: $\max_\lambda g(\lambda)$.

Chúng ta sẽ cụ thể hoá với $n=2$, chọn $c, a, b$ và so sánh nghiệm primal-dual.


In [ ]:
# Ví dụ đối ngẫu: n=2
c = np.array([1.0, -2.0])
a = np.array([2.0, 1.0])
b = 1.0

# x(lambda) = -(c + lambda a)
def x_of_lambda(lmbd: float) -> np.ndarray:
    return -(c + lmbd * a)

# g(lambda) = L(x(lambda), lambda) = 1/2||x||^2 + c^T x + lambda(a^T x - b)
# với x = x(lambda)
def g_dual(lmbd: float) -> float:
    x = x_of_lambda(lmbd)
    return 0.5 * np.dot(x, x) + np.dot(c, x) + lmbd * (np.dot(a, x) - b)

# maximize g(lambda) (concave) -> ta có thể tìm đạo hàm g'(lambda)=0
# Tính analytic: a^T x(lambda) - b = 0 => a^T (-(c + lambda a)) - b = 0
# => -(a^T c) - lambda * (a^T a) - b = 0 => lambda* = -(a^T c + b)/(a^T a)
lam_star = -(np.dot(a, c) + b) / np.dot(a, a)
x_star_dual = x_of_lambda(lam_star)

print('lambda* =', lam_star)
print('x*(dual-induced) =', x_star_dual)

# Kiểm tra ràng buộc a^T x = b
print('a^T x =', np.dot(a, x_star_dual), '(mong đợi ~ b =', b, ')')


In [ ]:
# Vẽ đồ thị g(lambda)
lmbd_grid = np.linspace(lam_star - 5, lam_star + 5, 400)
vals = np.array([g_dual(l) for l in lmbd_grid])

plt.figure(figsize=(6,3.8))
plt.plot(lmbd_grid, vals, label='g(lambda)')
plt.axvline(lam_star, color='r', linestyle='--', label=f'lambda*={lam_star:.3f}')
plt.xlabel('lambda'); plt.ylabel('g(lambda)')
plt.title('Đồ thị hàm đối ngẫu g(lambda)')
plt.legend(); plt.grid(True)
plt.show()


In [ ]:
import numpy as np


# ======================================================
# 1️⃣ HÀM GIẢI PRIMAL + DUAL CHO BÀI TOÁN TỔNG QUÁT
# ======================================================

def solve_primal_dual_quadratic(c, a, b):
    """
    Bài toán:
        minimize  f(x) = 1/2 ||x||^2 + c^T x
        subject to a^T x = b

    Input:
        c: array-like, shape (n,)
        a: array-like, shape (n,)
        b: scalar

    Output:
        x_star : nghiệm primal tối ưu
        lambda_star : nghiệm dual tối ưu
        p_star : giá trị tối ưu primal f(x*)
        d_star : giá trị tối ưu dual g(lambda*)
    """

    # ---- ép về numpy vector 1D ----
    c = np.atleast_1d(np.array(c, dtype=float))
    a = np.atleast_1d(np.array(a, dtype=float))
    b = float(b)

    if c.shape != a.shape:
        raise ValueError("c và a phải cùng kích thước")

    ata = a @ a
    if abs(ata) < 1e-12:
        raise ValueError("a phải khác 0 để ràng buộc a^T x = b có ý nghĩa")

    # ---- nghiệm dual λ* từ điều kiện feasibility a^T x(λ) = b ----
    # a^T(-c - λ a) = b  ⇒  -(a^T c) - λ (a^T a) = b
    # ⇒ λ* = -(b + a^T c) / (a^T a)
    lambda_star = -(b + a @ c) / ata

    # ---- nghiệm primal x* ----
    x_star = -c - lambda_star * a

    # ---- giá trị primal tối ưu ----
    p_star = 0.5 * (x_star @ x_star) + c @ x_star

    # ---- hàm dual g(λ) = L(x(λ), λ) ----
    def dual_value(lmbd):
        x_l = -c - lmbd * a
        L = 0.5 * (x_l @ x_l) + c @ x_l + lmbd * (a @ x_l - b)
        return L

    d_star = dual_value(lambda_star)

    return x_star, lambda_star, p_star, d_star, dual_value


# ======================================================
# 2️⃣ VÍ DỤ CỤ THỂ n = 2 (bạn có thể đổi tự do)
# ======================================================

if __name__ == "__main__":
    # Ví dụ: n = 2
    # f(x) = 1/2 ||x||^2 + c^T x,  x = (x1,x2)
    # c = (1, -2),  a = (2, 1),   a^T x = b = 3
    c = [1, -2]
    a = [2, 1]
    b = 3

    x_star, lambda_star, p_star, d_star, dual_value = solve_primal_dual_quadratic(c, a, b)

    print("===== KẾT QUẢ PRIMAL - DUAL =====")
    print("c =", c)
    print("a =", a)
    print("b =", b)
    print("\nNghiệm primal x* =", x_star)
    print("Nghiệm dual   λ* =", lambda_star)
    print("\nGiá trị primal tối ưu  p* = f(x*) =", p_star)
    print("Giá trị dual tối ưu    d* = g(λ*) =", d_star)
    print("\nKiểm tra strong duality (p* ?= d*):")
    print("p* - d* =", p_star - d_star)


In [ ]:
import sympy as sp
import numpy as np


def primal_dual_solver(f_expr, x_symbols, eq_constraints=None):
    """
    Solve:
        minimize f(x)
        subject to h_i(x) = 0
        
    Automatic:
        - build Lagrangian
        - compute dual function
        - find optimal x*, λ*
        - check strong duality
        
    INPUT:
        f_expr: sympy expression of f(x)
        x_symbols: list [x1, x2, ..., xn]
        eq_constraints: list [h1(x), h2(x), ...]
        
    WORKS for nD automatically
    """

    if eq_constraints is None:
        eq_constraints = []

    # --------------------
    # PREPARE SYMBOLS
    # --------------------

    x = sp.Matrix(x_symbols)
    n = len(x_symbols)

    m = len(eq_constraints)
    λ = sp.Matrix(sp.symbols(f"λ1:{m+1}", real=True))

    # --------------------
    # BUILD LAGRANGIAN
    # --------------------
    L = f_expr
    for i in range(m):
        L += λ[i] * eq_constraints[i]

    print("\n===== LAGRANGIAN =====")
    print("L(x, λ) =")
    print(sp.simplify(L))

    # --------------------
    # STATIONARITY ∂L/∂x=0
    # --------------------
    grad_L = sp.Matrix([sp.diff(L, var) for var in x_symbols])

    print("\n===== KKT STATIONARITY =====")
    for i, gi in enumerate(grad_L):
        print(f"∂L/∂x{i+1} = {sp.simplify(gi)} = 0")

    # --------------------
    # SOLVE KKT SYSTEM
    # --------------------
    equations = list(grad_L)

    # add primal feasibility
    for h in eq_constraints:
        equations.append(h)

    unknowns = list(x_symbols) + list(λ)

    sol = sp.solve(equations, unknowns, dict=True)

    if not sol:
        print("\n⚠ No KKT solution found")
        return
    
    sol = sol[0]

    # --------------------
    # EXTRACT PRIMAL x*
    # --------------------
    x_star = sp.Matrix([sol[v] for v in x_symbols])

    # --------------------
    # DUAL λ*
    # --------------------
    λ_star = sp.Matrix([sol[v] for v in λ]) if m>0 else sp.Matrix([])

    print("\n===== OPTIMAL SOLUTION =====")
    print("x* =", x_star)
    print("λ* =", λ_star)

    # --------------------
    # PRIMAL VALUE
    # --------------------
    p_star = f_expr.subs([(x_symbols[i], x_star[i]) for i in range(n)])
    p_star = sp.simplify(p_star)

    print("\n===== PRIMAL VALUE =====")
    print("p* = f(x*) =", p_star)

    # --------------------
    # BUILD DUAL FUNCTION
    # --------------------
    x_lambda = sp.solve(list(grad_L), list(x_symbols), dict=True)

    if x_lambda:

        xλ = x_lambda[0]

        L_lambda = L.subs([(x_symbols[i], xλ[x_symbols[i]]) 
                                  for i in range(n)])

        g_lambda = sp.simplify(L_lambda)

        if m > 0:
            d_star = g_lambda.subs([(λ[i], λ_star[i]) for i in range(m)])
            d_star = sp.simplify(d_star)

            print("\n===== DUAL FUNCTION g(λ) =====")
            print("g(λ) =", g_lambda)

            print("\n===== DUAL VALUE =====")
            print("g(λ*) =", d_star)

            print("\n===== STRONG DUALITY CHECK =====")
            print("p* - g(λ*) =", sp.simplify(p_star - d_star))

    print("\nDONE ✓\n")


In [ ]:
# sj x1 - 2*x2 = 0
x1, x2 = sp.symbols('x1 x2', real=True)

f = 0.5*(x1**2 + 4*x2**2) - x1 + 4*x2 + 2.5

h = x1 - 2*x2 - 2

primal_dual_solver(f, [x1, x2], [h])


In [ ]:
x1,x2,x3,x4 = sp.symbols('x1 x2 x3 x4')
f = 0.5*(x1**2+x2**2+x3**2+x4**2)

h1 = x1 + x2 - 3
h2 = x3 - x4

primal_dual_solver(f, [x1,x2,x3,x4], [h1, h2])


In [ ]:
import sympy as sp

def primal_dual_solver_general(f_expr, x_symbols, constraints):

    x = sp.Matrix(x_symbols)
    n = len(x)

    eq_cons = []
    ineq_cons = []
    ineq_sign = []  # store signs

    # ---------------------------
    # PREPROCESS CONSTRAINT INPUT
    # ---------------------------
    for ctype, expr in constraints:

        if ctype in ["="]:
            eq_cons.append(expr)

        elif ctype in ["<","<="]:  
            # convert to g(x) ≤ 0
            ineq_cons.append(expr)
            ineq_sign.append(+1)   # λ ≥ 0

        elif ctype in [">",">="]:
            # convert to (-expr ≤ 0)
            ineq_cons.append(-expr)
            ineq_sign.append(-1)   # λ ≤ 0

        else:
            raise ValueError("Unknown constraint symbol")

    # ---------------------------------
    # VARIABLES FOR MULTIPLIERS
    # ---------------------------------
    λ = sp.Matrix(sp.symbols(f"lam1:{len(ineq_cons)+1}", real=True))
    ν = sp.Matrix(sp.symbols(f"nu1:{len(eq_cons)+1}", real=True))

    # --------------------
    # BUILD LAGRANGIAN
    # --------------------
    L = f_expr

    for i,h in enumerate(eq_cons):
        L += ν[i]*h

    for j,g in enumerate(ineq_cons):
        L += λ[j]*g

    print("\n===== LAGRANGIAN =====")
    print(L)

    # --------------------
    # STATIONARITY
    # --------------------
    grad_L = sp.Matrix([sp.diff(L, xi) for xi in x_symbols])

    print("\n===== ∂L/∂x = 0 =====")
    for eq in grad_L:
        print(eq,"= 0")

    # --------------------
    # BUILD KKT EQUATIONS
    # --------------------
    eqs = list(grad_L)

    # primal eq feasibility
    for h in eq_cons:
        eqs.append(h)

    # complementary slackness
    for j,g in enumerate(ineq_cons):
        eqs.append(λ[j]*g)

    # variable list
    unknowns = list(x_symbols) + list(λ) + list(ν)

    # solve
    sol = sp.solve(eqs, unknowns, dict=True)

    if not sol:
        print("\n❌ No symbolic KKT solution found.")
        return

    sol = sol[0]

    # ---------------------------------
    # FILTER DUAL FEASIBILITY
    # ---------------------------------
    feasible_solutions = []

    for key,value in sol.items():
        pass

    for var,val in sol.items():
        pass

    # only primal-dual output
    x_star = sp.Matrix([sol[v] for v in x_symbols])

    print("\n===== x* =====")
    print(x_star)

    # dual multipliers
    lam_star = sp.Matrix([sol[v] for v in λ]) if len(λ)>0 else None
    nu_star  = sp.Matrix([sol[v] for v in ν]) if len(ν)>0 else None

    print("\n===== λ* =====")
    print(lam_star)

    print("\n===== ν* =====")
    print(nu_star)

    # primal value
    f_star = f_expr.subs([(x_symbols[i],x_star[i]) for i in range(n)])
    print("\n===== f(x*) =====")
    print(sp.simplify(f_star))

    # strong duality check only if convex + equality constraint only
    if len(ineq_cons)==0:
        # dual function
        xλ_sol = sp.solve(list(grad_L), list(x_symbols), dict=True)
        if xλ_sol:
            xλ = xλ_sol[0]
            Lλ = L.subs([(x_symbols[i],xλ[x_symbols[i]]) for i in range(n)])
            gλ = sp.simplify(Lλ)
            dual_val = gλ.subs([(ν[i],nu_star[i]) for i in range(len(nu_star))])
            print("\n===== strong duality =====")
            print("f* - g* =", sp.simplify(f_star-dual_val))


In [ ]:
x,y = sp.symbols('x y', real=True)
f = -x**2 - y**2 + 4*x +6*y
cons = [("<=", x + y -5),
        (">=", x),
         (">=", y)
        ]

primal_dual_solver_general(f, [x,y], cons)

In [ ]:
import sympy as sp

def solve_optimization(f_expr, x_symbols, constraints):
    """
    Giải bài toán tối ưu có ràng buộc bằng điều kiện KKT
    
    Input:
        f_expr: hàm mục tiêu (sympy expression)
        x_symbols: list các biến [x, y, z, ...]
        constraints: list các ràng buộc [("=", expr), ("<=", expr), (">=", expr), ...]
    
    Output: In ra màn hình nghiệm tối ưu x*, λ*, ν*, f(x*)
    """
    
    x = sp.Matrix(x_symbols)
    n = len(x)
    eq_cons, ineq_cons, ineq_sign = [], [], []
    
    # Phân loại ràng buộc
    for ctype, expr in constraints:
        if ctype == "=":
            eq_cons.append(expr)
        elif ctype in ["<", "<="]:
            ineq_cons.append(expr)
            ineq_sign.append(+1)  # λ ≥ 0
        elif ctype in [">", ">="]:
            ineq_cons.append(-expr)
            ineq_sign.append(-1)  # λ ≤ 0
    
    # Tạo biến nhân tử Lagrange
    m_ineq, m_eq = len(ineq_cons), len(eq_cons)
    λ = sp.Matrix(sp.symbols(f"lam1:{m_ineq+1}", real=True)) if m_ineq > 0 else sp.Matrix([])
    ν = sp.Matrix(sp.symbols(f"nu1:{m_eq+1}", real=True)) if m_eq > 0 else sp.Matrix([])
    
    # Xây dựng Lagrangian
    L = f_expr
    for i, h in enumerate(eq_cons):
        L += ν[i] * h
    for j, g in enumerate(ineq_cons):
        L += λ[j] * g
    
    print("\n" + "="*70)
    print("LAGRANGIAN")
    print("="*70)
    print(f"L = {L}\n")
    
    # Điều kiện stationarity
    grad_L = sp.Matrix([sp.diff(L, xi) for xi in x_symbols])
    print("="*70)
    print("STATIONARITY: ∇L = 0")
    print("="*70)
    for i, eq in enumerate(grad_L):
        print(f"∂L/∂{x_symbols[i]} = {eq} = 0")
    
    # Hệ phương trình KKT
    eqs = list(grad_L)
    for h in eq_cons:
        eqs.append(h)
    for j, g in enumerate(ineq_cons):
        eqs.append(λ[j] * g)
    
    unknowns = list(x_symbols) + list(λ) + list(ν)
    
    # Giải hệ KKT
    try:
        solutions = sp.solve(eqs, unknowns, dict=True)
    except:
        print("\n❌ Không giải được hệ KKT")
        return
    
    if not solutions:
        print("\n❌ Không tìm thấy nghiệm")
        return
    
    print(f"\n✓ Tìm thấy {len(solutions)} nghiệm ứng viên\n")
    
    # Lọc nghiệm khả thi
    feasible = []
    for idx, sol in enumerate(solutions):
        is_feasible = True
        
        # Kiểm tra dual feasibility (dấu của λ)
        for j in range(m_ineq):
            lam_val = sol.get(λ[j], 0)
            try:
                lam_numeric = complex(lam_val)
                if lam_numeric.imag != 0:
                    is_feasible = False
                    break
                lam_real = lam_numeric.real
                if ineq_sign[j] == +1 and lam_real < -1e-10:
                    is_feasible = False
                    break
                if ineq_sign[j] == -1 and lam_real > 1e-10:
                    is_feasible = False
                    break
            except:
                pass
        
        # Kiểm tra primal feasibility (g(x) ≤ 0)
        if is_feasible:
            for j, g in enumerate(ineq_cons):
                g_val = g.subs([(x_symbols[i], sol.get(x_symbols[i], 0)) for i in range(n)])
                try:
                    if float(g_val) > 1e-8:
                        is_feasible = False
                        break
                except:
                    pass
        
        if is_feasible:
            feasible.append(sol)
    
    if not feasible:
        print("❌ Không có nghiệm KKT khả thi")
        return
    
    # Chọn nghiệm tốt nhất (f nhỏ nhất)
    best_sol = feasible[0]
    best_f = None
    for sol in feasible:
        x_val = sp.Matrix([sol.get(x_symbols[i], 0) for i in range(n)])
        f_val = f_expr.subs([(x_symbols[i], x_val[i]) for i in range(n)])
        try:
            f_numeric = float(sp.re(f_val))
            if best_f is None or f_numeric < best_f:
                best_f = f_numeric
                best_sol = sol
        except:
            pass
    
    # Kết quả
    x_star = sp.Matrix([best_sol.get(v, 0) for v in x_symbols])
    lam_star = sp.Matrix([best_sol.get(v, 0) for v in λ]) if m_ineq > 0 else None
    nu_star = sp.Matrix([best_sol.get(v, 0) for v in ν]) if m_eq > 0 else None
    f_star = f_expr.subs([(x_symbols[i], x_star[i]) for i in range(n)])
    
    print("="*70)
    print("KẾT QUẢ")
    print("="*70)
    print(f"\nx* = {x_star}")
    if lam_star is not None:
        print(f"\nλ* = {lam_star}")
    if nu_star is not None:
        print(f"\nν* = {nu_star}")
    print(f"\nf(x*) = {sp.simplify(f_star)}")
    
    # Kiểm tra đối ngẫu mạnh (nếu chỉ có ràng buộc đẳng thức)
    if m_ineq == 0 and m_eq > 0:
        try:
            xν_sol = sp.solve(list(grad_L), list(x_symbols), dict=True)
            if xν_sol:
                xν = xν_sol[0]
                Lν = L.subs([(x_symbols[i], xν[x_symbols[i]]) for i in range(n)])
                gν = sp.simplify(Lν)
                dual_val = gν.subs([(ν[i], nu_star[i]) for i in range(len(nu_star))])
                gap = sp.simplify(f_star - dual_val)
                print(f"\nDuality gap: f* - g* = {gap}")
                try:
                    if abs(float(sp.re(gap))) < 1e-8:
                        print("✓ Đối ngẫu mạnh được thỏa mãn")
                except:
                    pass
        except:
            pass
    
    print("\n" + "="*70 + "\n")


# ============================================
# SỬ DỤNG
# ============================================
# x² - y² + 4x + 6y  s.t. x+y≤5, x≥0, y≥0

x, y = sp.symbols('x y', real=True)

f = -x**2 - y**2 + 4*x +6*y
cons = [("<=", x + y -5),
        (">=", x),
         (">=", y)
        ]

solve_optimization(f, [x, y], cons)




## Phần B — Phương pháp Newton với backtracking
Ta xét hàm Rosenbrock kinh điển $f(x,y) = (1-x)^2 + 100(y - x^2)^2$. Viết gradient, Hessian, sau đó cài đặt Newton với điều kiện Armijo (backtracking). So sánh quỹ đạo với Gradient Descent để cảm nhận cải thiện điều kiện hoá.


### Backtracking Line Search

Mục tiêu: tìm hệ số bước \( \alpha > 0 \) sao cho điều kiện Armijo được thỏa mãn.

$$
\alpha_{k+1} = \rho \, \alpha_k, \quad 0 < \rho < 1
$$

$$
f(x + \alpha d) \le f(x) + c \alpha \nabla f(x)^\top d, \quad 0 < c < 1
$$

---

### Newton Method

Tại bước \( k \):

$$
\nabla f(x_k) = g_k, \quad \nabla^2 f(x_k) = H_k
$$

Phương trình Newton:

$$
H_k p_k = -g_k \quad \Rightarrow \quad p_k = -H_k^{-1} g_k
$$

Cập nhật nghiệm:

$$
x_{k+1} = x_k + \alpha_k p_k
$$

Nếu \( H_k \) suy biến, dùng nghịch đảo giả:

$$
p_k = -H_k^{+} g_k
$$

Tiêu chí dừng:

$$
\|\nabla f(x_k)\| < \text{tol}
$$

---

### Gradient Descent (Hạ dốc)

Cập nhật theo hướng âm gradient:

$$
x_{k+1} = x_k - \eta \nabla f(x_k)
$$

Tiêu chí dừng:

$$
\|\nabla f(x_k)\| < \text{tol}
$$



In [ ]:
def rosenbrock(x: np.ndarray) -> float:
    X, Y = x[0], x[1]
    return (1 - X)**2 + 100.0 * (Y - X**2)**2

def grad_rosenbrock(x: np.ndarray) -> np.ndarray:
    X, Y = x[0], x[1]
    dfdx = -2*(1 - X) - 400*X*(Y - X**2)
    dfdy = 200*(Y - X**2)
    return np.array([dfdx, dfdy])

def hess_rosenbrock(x: np.ndarray) -> np.ndarray:
    X, Y = x[0], x[1]
    dxx = 2 - 400*(Y - 3*X**2)
    dxy = -400*X
    dyx = -400*X
    dyy = 200
    return np.array([[dxx, dxy], [dyx, dyy]])


In [ ]:
def backtracking_line_search(f: Callable[[np.ndarray], float],
                             grad: Callable[[np.ndarray], np.ndarray],
                             x: np.ndarray,
                             direction: np.ndarray,
                             alpha_init: float = 1.0,
                             rho: float = 0.5,
                             c: float = 1e-4) -> float:
    f_x = f(x)
    grad_x = grad(x)
    alpha = alpha_init
    while f(x + alpha * direction) > f_x + c * alpha * np.dot(grad_x, direction):
        alpha *= rho
        if alpha < 1e-12:
            break
    return alpha


def newton_method(f: Callable[[np.ndarray], float],
                  grad: Callable[[np.ndarray], np.ndarray],
                  hess: Callable[[np.ndarray], np.ndarray],
                  x0: np.ndarray,
                  tol: float = 1e-8,
                  max_iter: int = 50,
                  line_search: bool = True) -> Tuple[np.ndarray, list]:
    x = x0.astype(float).copy()
    hist = [x.copy()]
    for _ in range(max_iter):
        g = grad(x)
        H = hess(x)
        if np.linalg.norm(g) < tol:
            break
        try:
            p = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            # Nếu Hessian suy biến, dùng pseudo-inverse
            p = -np.linalg.pinv(H) @ g
        step = backtracking_line_search(f, grad, x, p) if line_search else 1.0
        x = x + step * p
        hist.append(x.copy())
    return x, hist


def gradient_descent(f: Callable[[np.ndarray], float],
                     grad: Callable[[np.ndarray], np.ndarray],
                     x0: np.ndarray,
                     lr: float = 1e-3,
                     max_iter: int = 5000,
                     tol: float = 1e-8) -> Tuple[np.ndarray, list]:
    x = x0.astype(float).copy()
    hist = [x.copy()]
    for _ in range(max_iter):
        g = grad(x)
        if np.linalg.norm(g) < tol:
            break
        x = x - lr * g
        hist.append(x.copy())
    return x, hist


In [ ]:
# Chạy thử Newton vs GD trên Rosenbrock
x0 = np.array([-1.2, 1.0])

x_newton, hist_newton = newton_method(rosenbrock, grad_rosenbrock, hess_rosenbrock, x0, max_iter=30)
x_gd, hist_gd = gradient_descent(rosenbrock, grad_rosenbrock, x0, lr=1e-3, max_iter=50000)

print('Newton x* ~', x_newton, 'f* ~', rosenbrock(x_newton))
print('GD     x* ~', x_gd, 'f* ~', rosenbrock(x_gd))


In [ ]:
# Vẽ quỹ đạo trên contour f
hist_newton_arr = np.array(hist_newton)
hist_gd_arr = np.array(hist_gd)

# Lưới để vẽ contour
xv = np.linspace(-2, 2, 400)
yv = np.linspace(-1, 2, 400)
XX, YY = np.meshgrid(xv, yv)
ZZ = (1-XX)**2 + 100*(YY-XX**2)**2

plt.figure(figsize=(6.5,5))
cs = plt.contour(XX, YY, np.log10(ZZ+1e-6), levels=30, cmap='viridis')
plt.clabel(cs, inline=True, fontsize=8, fmt='%.1f')
plt.plot(hist_gd_arr[:,0], hist_gd_arr[:,1], '-o', ms=2, lw=1, label='GD', alpha=0.8)
plt.plot(hist_newton_arr[:,0], hist_newton_arr[:,1], '-s', ms=4, lw=1.5, label='Newton', alpha=0.9)
plt.scatter([1],[1], c='red', s=50, label='Min (1,1)')
plt.legend(); plt.xlabel('x'); plt.ylabel('y')
plt.title('Quỹ đạo GD vs Newton trên Rosenbrock (contour log10(f))')
plt.grid(True, alpha=0.2)
plt.show()


In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# =========================================================
# BUILDING GENERAL OPTIMIZATION MODEL (1D → nD)
# =========================================================
def build_model(f_expr, variables):

    # symbolic → numeric lambdify
    f = sp.lambdify(variables, f_expr, "numpy")

    # gradient
    grad_syms = [sp.diff(f_expr, v) for v in variables]
    grad = sp.lambdify(variables, grad_syms, "numpy")

    # Hessian
    H_syms = sp.hessian(f_expr, variables)
    hess = sp.lambdify(variables, H_syms, "numpy")

    return f, grad, hess


# =========================================================
# NEWTON with BACKTRACKING ARMIJO
# =========================================================
def newton_armijo(f, grad, hess, x0, alpha=0.4, beta=0.8,
                  max_iter=200, tol=1e-6):

    x = np.array(x0, dtype=float)
    hist = [x.copy()]

    for _ in range(max_iter):

        g = np.array(grad(*x), dtype=float)
        H = np.array(hess(*x), dtype=float)

        # stopping
        if np.linalg.norm(g) < tol:
            break

        # Newton step
        try:
            p = np.linalg.solve(H, -g)
        except np.linalg.LinAlgError:       # fallback GD direction
            p = -g  

        # Armijo line search
        t = 1
        fx = f(*x)
        while f(*(x + t*p)) > fx + alpha*t*np.dot(g, p):
            t *= beta

        x = x + t*p
        hist.append(x.copy())

    return np.array(hist)


# =========================================================
# GRADIENT DESCENT + BACKTRACKING ARMIJO
# =========================================================
def gd_armijo(f, grad, x0, alpha=0.4, beta=0.8,
              max_iter=200, tol=1e-6):

    x = np.array(x0, dtype=float)
    hist = [x.copy()]

    for _ in range(max_iter):

        g = np.array(grad(*x), dtype=float)

        if np.linalg.norm(g) < tol:
            break

        p = -g

        t = 1
        fx = f(*x)
        while f(*(x + t*p)) > fx + alpha*t*np.dot(g, p):
            t *= beta

        x = x + t*p
        hist.append(x.copy())

    return np.array(hist)


# =========================================================
# PLOTTING IN 2D CASE ONLY
# =========================================================
def plot_2D(f, newton_path, gd_path):
    xs = np.linspace(-2,2,400)
    ys = np.linspace(-1,3,400)
    X,Y = np.meshgrid(xs,ys)
    Z = f(X,Y)

    plt.figure(figsize=(8,6))
    plt.contour(X,Y,Z,levels=40)

    plt.plot(newton_path[:,0],newton_path[:,1],'ro--',label="Newton")
    plt.plot(gd_path[:,0],gd_path[:,1],'go--',label="Gradient Descent")

    plt.legend()
    plt.title("Path comparison")
    plt.show()


# =========================================================
# GENERAL RUN FUNCTION: Just change f_expr and variables
# =========================================================
def run(f_expr, variables, x0):

    f, grad, hess = build_model(f_expr, variables)

    newton_hist = newton_armijo(f, grad, hess, x0)
    gd_hist     = gd_armijo(f, grad, x0)

    print("\nFinal Newton point =", newton_hist[-1])
    print("Final GD point     =", gd_hist[-1])

    # only visualizable when dim = 2
    if len(variables)==2:
        plot_2D(f,newton_hist,gd_hist)


# =========================================================
# DEFAULT TEST: ROSENBROCK
# =========================================================
if __name__ == "__main__":

    x, y = sp.symbols("x y", real=True)

    f_expr = (1-x)**2 + 100*(y - x**2)**2

    run(f_expr, [x,y], x0=[-1.2, 1.0])


## Phần B — Ví dụ bổ sung: Newton có/không backtracking trên Rosenbrock
Để minh hoạ vai trò của backtracking (điều kiện Armijo) trong phương pháp Newton, ta so sánh Newton với backtracking và Newton full-step (không backtracking) từ một điểm khởi tạo khác trên hàm Rosenbrock, đồng thời hiển thị quỹ đạo và giá trị hàm mục tiêu để thấy sự khác biệt về độ ổn định và tốc độ hội tụ.


In [ ]:
# Ví dụ bổ sung Phần B: Newton có/không backtracking vs GD trên Rosenbrock
# Dùng khởi tạo khác để quan sát khác biệt ổn định
x0_alt = np.array([-1.8, 1.8])

# Newton với backtracking (đã có hàm newton_method)
x_newton_bt, hist_newton_bt = newton_method(rosenbrock, grad_rosenbrock, hess_rosenbrock,
                                            x0_alt, max_iter=40, line_search=True)

# Newton full-step (không backtracking): tận dụng newton_method với line_search=False
x_newton_fs, hist_newton_fs = newton_method(rosenbrock, grad_rosenbrock, hess_rosenbrock,
                                            x0_alt, max_iter=40, line_search=False)

# Gradient Descent giữ nguyên lr như trước để tham chiếu
x_gd_alt, hist_gd_alt = gradient_descent(rosenbrock, grad_rosenbrock, x0_alt, lr=1e-3, max_iter=80000)

print('Newton BT  x* ~', x_newton_bt, 'f* ~', rosenbrock(x_newton_bt))
print('Newton FS  x* ~', x_newton_fs, 'f* ~', rosenbrock(x_newton_fs))
print('GD (alt)   x* ~', x_gd_alt,    'f* ~', rosenbrock(x_gd_alt))

# Vẽ quỹ đạo
h_bt = np.array(hist_newton_bt)
h_fs = np.array(hist_newton_fs)
h_gd = np.array(hist_gd_alt)

xv = np.linspace(-2.2, 2.2, 450)
yv = np.linspace(-1.2, 2.2, 450)
XX, YY = np.meshgrid(xv, yv)
ZZ = (1-XX)**2 + 100*(YY-XX**2)**2

plt.figure(figsize=(6.8,5.4))
cs = plt.contour(XX, YY, np.log10(ZZ+1e-6), levels=35, cmap='viridis')
plt.clabel(cs, inline=True, fontsize=8, fmt='%.1f')
plt.plot(h_gd[:,0], h_gd[:,1], '-o', ms=2, lw=1, label='GD (alt)', alpha=0.75)
plt.plot(h_bt[:,0], h_bt[:,1], '-s', ms=4, lw=1.4, label='Newton + backtracking', alpha=0.9)
plt.plot(h_fs[:,0], h_fs[:,1], '-^', ms=4, lw=1.2, label='Newton full-step', alpha=0.9)
plt.scatter([1],[1], c='red', s=50, label='Min (1,1)')
plt.legend(); plt.xlabel('x'); plt.ylabel('y')
plt.title('Phần B (bổ sung): Quỹ đạo Newton có/không backtracking vs GD trên Rosenbrock')
plt.grid(True, alpha=0.25)
plt.show()

# Vẽ tiến triển giá trị hàm mục tiêu theo iteration
plt.figure(figsize=(6.2,3.6))
f_bt = [rosenbrock(p) for p in h_bt]
f_fs = [rosenbrock(p) for p in h_fs]
f_gd = [rosenbrock(p) for p in h_gd]
plt.semilogy(f_bt, label='Newton + backtracking')
plt.semilogy(f_fs, label='Newton full-step')
plt.semilogy(f_gd, label='GD (alt)')
plt.xlabel('Iteration'); plt.ylabel('f(x) (log scale)')
plt.title('Phần B (bổ sung): Tiến triển f(x)')
plt.legend(); plt.grid(True, which='both', alpha=0.3)
plt.show()


In [ ]:
import sympy as sp
import numpy as np


# =================================================================
# BUILD GENERAL MODEL FROM SYMBOLIC: f, grad f, Hessian f
# =================================================================
def build_function(f_expr, variables):
    """
    Input:
        f_expr: sympy objective function
        variables: list of symbolic variables [x1,x2,...,xn]

    Output:
        f(x), grad(x), Hessian(x) (numpy-callable)
    """

    f   = sp.lambdify(variables, f_expr, "numpy")

    grad_expr = [sp.diff(f_expr, v) for v in variables]
    grad = sp.lambdify(variables, grad_expr, "numpy")

    hess_expr = sp.hessian(f_expr, variables)
    hess = sp.lambdify(variables, hess_expr, "numpy")

    return f, grad, hess


# =================================================================
# NEWTON WITH BACKTRACKING
# =================================================================
def newton_backtracking(
    f, grad, hess,
    x0,
    alpha = 0.4,
    beta  = 0.8,
    tol   = 1e-6,
    max_iter = 200,
    verbose = True
):
    """
    Run Newton method with Armijo line search.
    
    Input:
        f, grad, hess : callables
        x0   : initial guess (scalar or vector)
        alpha, beta: Armijo params
        tol  : tolerance
        max_iter: max loops
    
    Output:
        history: list of x_k
    """

    # ensure vector format
    x = np.array(x0, dtype=float)
    if x.ndim == 0:
        x = x.reshape(1)

    hist = [x.copy()]

    for k in range(max_iter):

        g = np.asarray(grad(*x), dtype=float)

        # stopping condition
        if np.linalg.norm(g) < tol:
            if verbose:
                print(f"STOP: gradient norm small at iter {k}")
            break

        H = np.asarray(hess(*x), dtype=float)

        # Newton direction
        try:
            p = np.linalg.solve(H, -g)
        except np.linalg.LinAlgError:   # if Hessian singular
            p = -g                      # fallback gradient descent

        # Armijo backtracking
        t = 1.0
        fx = f(*x)

        while True:
            x_trial = x + t*p
            f_trial = f(*x_trial)

            if f_trial <= fx + alpha*t*np.dot(g,p):
                break

            t *= beta

            if t < 1e-12:
                break

        x = x + t*p
        hist.append(x.copy())

        if verbose:
            print(f"Iter {k:3d} | f={fx:.6f} | step={t:.3e} | x={x}")

    return np.array(hist)


# =================================================================
# UNIVERSAL RUN FUNCTION — CHANGE ONLY f_expr + x0
# =================================================================
def run_newton(f_expr, variables, x0, **kwargs):
    """
    Build model & run Newton easily:
    """

    f, grad, hess = build_function(f_expr, variables)

    hist = newton_backtracking(f, grad, hess, x0, **kwargs)

    print("\nFinal x* =", hist[-1])
    print("Final f* =", f(*hist[-1]))
    print("Iterations =", len(hist)-1)

    return hist, f, grad, hess


In [ ]:
x, y = sp.symbols("x y", real=True)
f_expr = (1-x)**2 + 100*(y - x**2)**2

run_newton(f_expr, [x,y], x0=[-1.2, 1])


In [ ]:
x1,x2,x3 = sp.symbols("x1 x2 x3")

f_expr = x1**2 + x2**2 + 5*x3**2 - 3*x2 + x1*x3

run_newton(f_expr, [x1,x2,x3], x0=[1,2,3])


In [ ]:
x = sp.Symbol("x", real=True)

f_expr = sp.sin(x) + x**4

run_newton(f_expr, [x], x0=[2])


## Phần C — Bài tập hàm đơn giản
Ba bài này giúp luyện Newton trên các hàm cơ bản: 1D bậc bốn (lồi), quadratic trục chuẩn 2D


In [ ]:
# Newton 1D với backtracking
from typing import Callable

def backtracking_1d(f: Callable[[float], float], df: Callable[[float], float], x: float,
                    p: float, alpha_init: float = 1.0, rho: float = 0.5, c: float = 1e-4) -> float:
    fx = f(x)
    g = df(x)
    alpha = alpha_init
    while f(x + alpha * p) > fx + c * alpha * g * p:
        alpha *= rho
        if alpha < 1e-12:
            break
    return alpha


def newton_1d(f: Callable[[float], float], df: Callable[[float], float], d2f: Callable[[float], float],
              x0: float, max_iter: int = 50, tol: float = 1e-8) -> tuple[float, list]:
    x = float(x0)
    hist = [x]
    for _ in range(max_iter):
        g = df(x)
        if abs(g) < tol:
            break
        H = d2f(x)
        # tránh chia cho 0, thêm đệm nhỏ nếu cần
        if abs(H) < 1e-12:
            H = 1e-12 if H >= 0 else -1e-12
        p = - g / H
        alpha = backtracking_1d(f, df, x, p)
        x = x + alpha * p
        hist.append(x)
    return x, hist


### Bài: Newton 1D trên hàm bậc bốn
Xét $f(x) = (x-2)^4 + 0.5\,(x+1)^2$. Viết đạo hàm bậc nhất, bậc hai và chạy Newton từ nhiều điểm khởi tạo. Vẽ đồ thị 1D kèm các điểm lặp.


In [ ]:
# Bài 4
f4 = lambda x: (x-2.0)**4 + 0.5*(x+1.0)**2
df4 = lambda x: 4*(x-2.0)**3 + (x+1.0)
d2f4 = lambda x: 12*(x-2.0)**2 + 1.0

inits = [-4.0, 0.0, 4.0]
traj_4 = []
for x0 in inits:
    x_star, hist = newton_1d(f4, df4, d2f4, x0)
    traj_4.append((x0, x_star, hist))
    print(f'Init {x0:5.1f} -> x*={x_star:.6f}, f*={f4(x_star):.6e}, iters={len(hist)-1}')

# Vẽ
xs = np.linspace(-5, 6, 600)
ys = np.array([f4(x) for x in xs])
plt.figure(figsize=(6.5,3.8))
plt.plot(xs, ys, label='f4(x)')
for x0, x_star, hist in traj_4:
    pts_x = np.array(hist)
    pts_y = np.array([f4(x) for x in pts_x])
    plt.plot(pts_x, pts_y, '-o', ms=3, lw=1, label=f'Newton from {x0}')
plt.xlabel('x'); plt.ylabel('f(x)'); plt.title('Bài 4: Newton 1D trên hàm bậc bốn')
plt.legend(); plt.grid(True, alpha=0.2)
plt.show()


In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# ============================================================
# BUILD FUNCTION MODEL (GENERAL 1D → nD)
# ============================================================
def build_function(f_expr, variables):

    f = sp.lambdify(variables, f_expr, "numpy")

    grad_expr = [sp.diff(f_expr, v) for v in variables]
    grad = sp.lambdify(variables, grad_expr, "numpy")

    hess_expr = sp.hessian(f_expr, variables)
    hess = sp.lambdify(variables, hess_expr, "numpy")

    return f, grad, hess, grad_expr, hess_expr



# ============================================================
# NEWTON METHOD FOR GENERAL DIMENSION
# ============================================================
def newton(f, grad, hess, x0, tol=1e-8, max_iter=50):

    x = np.array(x0, dtype=float)

    # unify to vector form
    if x.ndim == 0:
        x = x.reshape(1)

    hist = [x.copy()]

    for _ in range(max_iter):

        g = np.asarray(grad(*x), dtype=float)
        H = np.asarray(hess(*x), dtype=float)

        if g.ndim == 0:
            g = np.array([g])
        if H.ndim == 0:
            H = np.array([[H]])

        if np.linalg.norm(g) < tol:
            break

        try:
            p = np.linalg.solve(H, -g)
        except np.linalg.LinAlgError:
            p = -g

        x = x + p
        hist.append(x.copy())

    # return list of scalars for 1D, vectors for nD
    if len(x) == 1:
        return [float(v[0]) for v in hist]  
    else:
        return np.array(hist)



# ============================================================
# PLOT FOR 1D CASE ONLY
# ============================================================
def plot_1D(f, trajectories, x_range=(-4,4)):

    xs = np.linspace(x_range[0], x_range[1], 500)
    ys = f(xs)

    plt.figure(figsize=(8,5))
    plt.plot(xs, ys, label="f(x)", color="black")

    colors = ["red","green","blue","purple"]

    for i,(x0,hist) in enumerate(trajectories.items()):
        c = colors[i%len(colors)]

        plt.plot(hist, f(np.array(hist)), "o--", color=c,
                 label=f"x0={x0} → {hist[-1]:.4f}")

    plt.title("Newton iteration paths 1D")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.grid(True)
    plt.show()



# ============================================================
# MAIN DRIVER FUNCTION
# ============================================================
def run_newton(f_expr, variables, x0_list):

    f, grad, hess, grad_expr, hess_expr = build_function(f_expr, variables)

    print("\n==== First derivative ====")
    for g in grad_expr:
        print(g)

    print("\n==== Second derivative ====")
    print(hess_expr)

    results = {}

    for x0 in x0_list:

        hist = newton(f, grad, hess, x0)

        results[x0] = hist

        print(f"\nInit x0={x0}  →  x*={hist[-1]}   iter={len(hist)-1}")

    # plot only if 1D
    if len(variables)==1:
        plot_1D(f, results)

    return results
x = sp.Symbol("x")
f_expr = (x-2)**4 + 0.5*(x+1)**2

run_newton(
    f_expr,
    [x],
    x0_list=[-4, 0, 4]
)


In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# ===============================================================
# BUILD FUNCTION MODEL (GENERAL 1D → nD)
# ===============================================================
def build_function(f_expr, variables):

    f = sp.lambdify(variables, f_expr, "numpy")

    grad_expr = [sp.diff(f_expr, v) for v in variables]
    grad = sp.lambdify(variables, grad_expr, "numpy")

    hess_expr = sp.hessian(f_expr, variables)
    hess = sp.lambdify(variables, hess_expr, "numpy")

    return f, grad, hess, grad_expr, hess_expr


# ===============================================================
# BACKTRACKING LINE SEARCH (GENERAL nD)
# ===============================================================
def backtracking(f, grad, x, p, alpha0=1.0, rho=0.5, c=1e-4):

    fx = f(*x) if len(x)>1 else f(x[0])
    g  = grad(*x) if len(x)>1 else np.array([grad(x[0])])

    alpha = alpha0

    while True:
        x_new = x + alpha * p

        f_new = f(*x_new) if len(x)>1 else f(x_new[0])

        if f_new <= fx + c * alpha * np.dot(g, p):
            break

        alpha *= rho

        if alpha < 1e-12:
            break

    return alpha


# ===============================================================
# NEWTON + BACKTRACKING
# ===============================================================
def newton_backtracking(f, grad, hess, x0, tol=1e-8, max_iter=50):

    x = np.array(x0, dtype=float)
    x = x.reshape(-1)   # force vector format

    hist = [x.copy()]

    for _ in range(max_iter):

        g = np.asarray(grad(*x), dtype=float)
        H = np.asarray(hess(*x), dtype=float)

        # 1D special handling
        if g.ndim == 0:
            g = np.array([g])
        if H.ndim == 0:
            H = np.array([[H]])

        if np.linalg.norm(g) < tol:
            break

        try:
            p = -np.linalg.solve(H, g)
        except:
            p = -g

        alpha = backtracking(f, grad, x, p)

        x = x + alpha*p

        hist.append(x.copy())

    return np.array(hist)


# ===============================================================
# PLOTTING FOR 1D
# ===============================================================
def plot_1D(f, results, x_range=(-4,4)):

    xs = np.linspace(x_range[0], x_range[1], 400)
    ys = f(xs)

    plt.figure(figsize=(8,5))
    plt.plot(xs, ys, label="f(x)", color="black")

    colors = ["red","green","blue","purple"]

    for i,(x0,hist) in enumerate(results.items()):

        c = colors[i%len(colors)]

        xs_hist = hist.flatten()
        plt.plot(xs_hist, f(xs_hist), "o--", color=c,
                 label=f"x0={x0} → {hist[-1][0]:.4f}")

    plt.title("Newton + Backtracking Path (1D)")
    plt.legend()
    plt.grid(True)
    plt.show()


# ===============================================================
# MAIN WRAPPER
# ===============================================================
def run_newton_backtracking(f_expr, variables, x0_list):

    f, grad, hess, grad_expr, hess_expr = build_function(f_expr, variables)

    print("\n==== First derivative ====")
    for g in grad_expr:
        print(g)

    print("\n==== Hessian ====")
    print(hess_expr)

    results = {}

    for x0 in x0_list:

        hist = newton_backtracking(f, grad, hess, x0)

                # build a usable dictionary key
        if np.isscalar(x0):
            key = x0                      # use raw number
        else:
            key = tuple(x0)              # use tuple
        
        results[key] = hist


        print(f"\nInit x0={x0}  →  x*={hist[-1]}   iter={len(hist)-1}")

    if len(variables)==1:
        plot_1D(f, results)

    return results
x = sp.Symbol("x")

f_expr = (x-2)**4 + 0.5*(x+1)**2

run_newton_backtracking(
    f_expr,
    [x],
    x0_list=[-4, 0, 4]
)


$$f(x_1, x_2, \dots, x_6) = \sum_{i=1}^{5} \left[ 100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2 \right]$$

In [ ]:
# ==========================
# 6-D ROSENBROCK EXAMPLE
# ==========================

# 6 symbolic variables
x1,x2,x3,x4,x5,x6 = sp.symbols("x1 x2 x3 x4 x5 x6", real=True)

variables = [x1,x2,x3,x4,x5,x6]

# generalized rosenbrock
f_expr = 0
for i in range(5):
    xi   = variables[i]
    xip1 = variables[i+1]
    f_expr += 100*(xip1 - xi**2)**2 + (1 - xi)**2

# initial guesses
x0_list = [
    [-1.5,  1.5, -1,  2, -2,  1],   # difficult start
    [0.5,   0.5, 0.5, 0.5, 0.5, 0.5], 
    [3,    -2,   1,  -1,   2, -1],
]

results = run_newton_backtracking(
    f_expr,
    variables,
    x0_list=x0_list
)


### Bài 5: Quadratic trục chuẩn 2D
Xét $f(x)=\tfrac{1}{2}x^\top Q x + q^\top x$ với $Q=\operatorname{diag}(1, 50)$ và $q=(-2,1)$. So sánh Newton (H=Q) và GD, vẽ contour và quỹ đạo.


In [ ]:

# Bài 5
Qd = np.diag([1.0, 50.0])
qd = np.array([-2.0, 1.0])
fd = lambda x: 0.5 * x @ Qd @ x + qd @ x
gd = lambda x: Qd @ x + qd
Hd = lambda x: Qd

x0 = np.array([3.0, -3.0])
xN, hN = newton_method(fd, gd, Hd, x0, max_iter=10)
xG, hG = gradient_descent(fd, gd, x0, lr=5e-3, max_iter=20000)
print('Diag Quad: Newton x* ~', xN, 'f* ~', fd(xN))
print('Diag Quad: GD     x* ~', xG, 'f* ~', fd(xG))

hN = np.array(hN); 
hG = np.array(hG)
xv = np.linspace(-4, 4, 300)
yv = np.linspace(-4, 4, 300)
XX, YY = np.meshgrid(xv, yv)
ZZ = 0.5*(Qd[0,0]*XX**2 + 2*Qd[0,1]*XX*YY + Qd[1,1]*YY**2) + qd[0]*XX + qd[1]*YY

plt.figure(figsize=(6.2,5))
cs = plt.contour(XX, YY, ZZ, levels=30, cmap='viridis')
plt.clabel(cs, inline=True, fontsize=8)
plt.plot(hG[:,0], hG[:,1], '-o', ms=2, lw=1, label='GD')
plt.plot(hN[:,0], hN[:,1], '-s', ms=4, lw=1.5, label='Newton')
plt.legend(); plt.xlabel('x'); plt.ylabel('y'); plt.title('Bài 5: Quadratic trục chuẩn — GD vs Newton')
plt.grid(True, alpha=0.2)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================================================
# 1. ĐỊNH NGHĨA HÀM QUADRATIC TỔNG QUÁT
#     f(x) = 1/2 x^T Q x + q^T x
# ==========================================================

def quad_f(Q: np.ndarray, q: np.ndarray, x: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    return 0.5 * x @ Q @ x + q @ x

def quad_grad(Q: np.ndarray, q: np.ndarray, x: np.ndarray) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    return Q @ x + q

def quad_hess(Q: np.ndarray) -> np.ndarray:
    # Hessian của quadratic là Q (hằng)
    return Q


# ==========================================================
# 2. NEWTON (H = Q CỐ ĐỊNH)
# ==========================================================

def newton_quadratic(Q, q, x0, tol=1e-8, max_iter=20):
    """
    Newton cho hàm quadratic f(x) = 1/2 x^T Q x + q^T x.
    Với Q đối xứng xác định dương → hội tụ 1 bước từ mọi x0.
    Nhưng ta vẫn lặp để xem quỹ đạo.
    """
    x = np.asarray(x0, dtype=float)
    Q = np.asarray(Q, dtype=float)
    q = np.asarray(q, dtype=float)

    H = quad_hess(Q)
    hist = [x.copy()]

    for k in range(max_iter):
        g = quad_grad(Q, q, x)
        if np.linalg.norm(g) < tol:
            break

        # Bước Newton: Q p = -g  →  p = -Q^{-1} g
        p = -np.linalg.solve(H, g)

        x = x + p
        hist.append(x.copy())

    return np.array(hist)


# ==========================================================
# 3. GRADIENT DESCENT (CỐ ĐỊNH LEARNING RATE)
# ==========================================================

def gradient_descent_quadratic(Q, q, x0, learning_rate=0.03, tol=1e-8, max_iter=200000):
    """
    GD cho quadratic. 
    Với step size α < 2 / L (L = eigenvalue max của Q) thì hội tụ.
    """
    x = np.asarray(x0, dtype=float)
    Q = np.asarray(Q, dtype=float)
    q = np.asarray(q, dtype=float)

    hist = [x.copy()]

    for k in range(max_iter):
        g = quad_grad(Q, q, x)

        if np.linalg.norm(g) < tol:
            break

        x = x - learning_rate * g
        hist.append(x.copy())

    return np.array(hist)


# ==========================================================
# 4. VẼ CONTOUR VÀ QUỸ ĐẠO (CHỈ DÙNG KHI dim = 2)
# ==========================================================

def plot_quadratic_2d(Q, q, newton_path, gd_path):
    assert Q.shape == (2,2), "Contour 2D chỉ hỗ trợ Q 2x2"
    assert q.shape == (2,), "q phải có chiều 2 cho plot 2D"

    # Lấy tất cả điểm để chọn vùng vẽ
    all_points = np.vstack([newton_path, gd_path])
    x_min, x_max = all_points[:,0].min() - 1, all_points[:,0].max() + 1
    y_min, y_max = all_points[:,1].min() - 1, all_points[:,1].max() + 1

    xs = np.linspace(x_min, x_max, 200)
    ys = np.linspace(y_min, y_max, 200)
    X, Y = np.meshgrid(xs, ys)

    # f(x) = 1/2(x^T Q x) + q^T x, với x = (X, Y)
    Z = (
        0.5 * (
            Q[0,0]*X**2 + 2*Q[0,1]*X*Y + Q[1,1]*Y**2
        )
        + q[0]*X + q[1]*Y
    )

    plt.figure(figsize=(7,6))
    cs = plt.contour(X, Y, Z, levels=30)
    plt.clabel(cs, inline=True, fontsize=8, fmt="%.1f")

    # Path Newton
    plt.plot(newton_path[:,0], newton_path[:,1], 'ro--', label='Newton')

    # Path GD
    plt.plot(gd_path[:,0], gd_path[:,1], 'go--', label='Gradient Descent')

    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Contour f(x) và quỹ đạo Newton vs GD')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


# ==========================================================
# 5. HÀM ĐIỀU PHỐI TỔNG QUÁT
# ==========================================================

def run_quadratic_demo(Q, q, x0, gd_lr=0.03, gd_max_iter=20000):
    """
    Chạy full:
      - tính nghiệm tối ưu x*
      - Newton path (H=Q)
      - GD path
      - nếu dim=2 → vẽ contour + path
      - nếu dim>2 → vẽ f(x_k) theo iteration
    """
    Q = np.asarray(Q, dtype=float)
    q = np.asarray(q, dtype=float)
    x0 = np.asarray(x0, dtype=float)

    dim = Q.shape[0]
    assert Q.shape == (dim, dim)
    assert q.shape == (dim,)

    # nghiệm tối ưu analytic: Q x* + q = 0 → x* = -Q^{-1} q
    x_star = -np.linalg.solve(Q, q)
    print("x* (analytic) =", x_star)
    print("f(x*) =", quad_f(Q, q, x_star))

    # Newton
    newton_path = newton_quadratic(Q, q, x0)
    print(f"\nNewton: iters = {len(newton_path)-1}")
    print("  last x =", newton_path[-1])
    print("  f(last) =", quad_f(Q, q, newton_path[-1]))

    # GD
    gd_path = gradient_descent_quadratic(Q, q, x0, learning_rate=gd_lr, max_iter=gd_max_iter)
    print(f"\nGD: iters = {len(gd_path)-1}")
    print("  last x =", gd_path[-1])
    print("  f(last) =", quad_f(Q, q, gd_path[-1]))

    # Plot
    if dim == 2:
        plot_quadratic_2d(Q, q, newton_path, gd_path)
    else:
        # vẽ f(x_k) theo iteration cho nD
        f_newton = [quad_f(Q,q,x) for x in newton_path]
        f_gd     = [quad_f(Q,q,x) for x in gd_path]

        plt.figure(figsize=(7,4))
        plt.semilogy(f_newton, 'ro--', label='Newton')
        plt.semilogy(f_gd, 'go--', label='GD')
        plt.xlabel('Iteration')
        plt.ylabel('f(x_k) (log scale)')
        plt.title('Convergence on quadratic (nD)')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

    return newton_path, gd_path, x_star
# ===== BÀI 5: QUADRATIC TRỤC CHUẨN 2D =====
Q = np.diag([1.0, 50.0])
q = np.array([-2.0, 1.0])

x0 = np.array([3.0, -3.0])   # bạn đổi thoải mái để xem quỹ đạo khác
gd_lr = 5e-3             # < 2/L = 2/50 = 0.04 → hội tụ

newton_path, gd_path, x_star = run_quadratic_demo(Q, q, x0, gd_lr=gd_lr, gd_max_iter=20000)


$$f(\mathbf{x}) = 10n + \sum_{i=1}^{n} \left[ x_i^2 - 10\cos(2\pi x_i) \right]$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def rosenbrock(x):
    """
    f(x) = sum_{i=1}^{n-1} [ (1-x_i)^2 + 100(x_{i+1}-x_i^2)^2 ]
    """
    x = np.asarray(x)
    return np.sum((1 - x[:-1])**2 + 100*(x[1:] - x[:-1]**2)**2)


def grad_rosenbrock(x):
    x = np.asarray(x)
    n = len(x)
    g = np.zeros(n)

    for i in range(n):
        if i > 0:
            g[i] += 200*(x[i] - x[i-1]**2)
        if i < n-1:
            g[i] += -2*(1-x[i]) - 400*x[i]*(x[i+1]-x[i]**2)
    return g


def hess_rosenbrock(x):
    x = np.asarray(x)
    n = len(x)
    H = np.zeros((n, n))

    for i in range(n):
        if i < n-1:
            H[i,i] += 2 - 400*(x[i+1]-x[i]**2) + 800*x[i]**2
            H[i,i+1] += -400*x[i]
        if i > 0:
            H[i,i] += 200
            H[i,i-1] += -400*x[i-1]
    return H


In [ ]:
def backtracking(f, grad, x, p, alpha0=1.0, rho=0.5, c=1e-4):
    alpha = alpha0
    fx = f(x)
    g = grad(x)

    while f(x + alpha*p) > fx + c*alpha*np.dot(g, p):
        alpha *= rho
        if alpha < 1e-12:
            break
    return alpha


In [ ]:
def newton_method(f, grad, hess, x0, tol=1e-6, max_iter=50, backtrack=True):
    x = x0.copy()
    hist_x = [x.copy()]
    hist_f = [f(x)]

    for _ in range(max_iter):
        g = grad(x)
        if np.linalg.norm(g) < tol:
            break

        H = hess(x)
        try:
            p = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            p = -np.linalg.pinv(H) @ g

        alpha = backtracking(f, grad, x, p) if backtrack else 1.0
        x = x + alpha*p

        hist_x.append(x.copy())
        hist_f.append(f(x))

    return np.array(hist_x), np.array(hist_f)


In [ ]:
def gradient_descent(f, grad, x0, lr=1e-3, tol=1e-6, max_iter=20000):
    x = x0.copy()
    hist_x = [x.copy()]
    hist_f = [f(x)]

    for _ in range(max_iter):
        g = grad(x)
        if np.linalg.norm(g) < tol:
            break
        x = x - lr*g
        hist_x.append(x.copy())
        hist_f.append(f(x))

    return np.array(hist_x), np.array(hist_f)


In [ ]:
# ====== INPUT ======
n = 2                      # đổi n tùy ý
x0 = np.array([-1.2, 1.0]) # điểm khởi tạo chuẩn Rosenbrock

# Newton có backtracking
xn_bt, fn_bt = newton_method(
    rosenbrock, grad_rosenbrock, hess_rosenbrock,
    x0, backtrack=True
)

# Newton full-step (không backtracking)
xn_full, fn_full = newton_method(
    rosenbrock, grad_rosenbrock, hess_rosenbrock,
    x0, backtrack=False
)

# Gradient Descent
xg, fg = gradient_descent(
    rosenbrock, grad_rosenbrock, x0, lr=1e-3
)

print("Newton + BT iters:", len(fn_bt)-1)
print("Newton full iters:", len(fn_full)-1)
print("GD iters:", len(fg)-1)


In [ ]:
plt.figure(figsize=(7,4))
plt.semilogy(fn_bt, label="Newton + Backtracking")
plt.semilogy(fn_full, label="Newton full-step")
plt.semilogy(fg, label="Gradient Descent")
plt.xlabel("Iteration")
plt.ylabel("f(x)")
plt.title("Convergence on Rosenbrock")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
if n == 2:
    x = np.linspace(-2, 2, 400)
    y = np.linspace(-1, 3, 400)
    X, Y = np.meshgrid(x, y)
    Z = (1-X)**2 + 100*(Y-X**2)**2

    plt.figure(figsize=(6,5))
    plt.contour(X, Y, Z, levels=30)
    plt.plot(xn_bt[:,0], xn_bt[:,1], 'o-', label="Newton + BT")
    plt.plot(xn_full[:,0], xn_full[:,1], 's--', label="Newton full")
    plt.plot(xg[:,0], xg[:,1], 'x-', label="GD")
    plt.legend()
    plt.title("Trajectories on Rosenbrock")
    plt.grid(True)
    plt.show()


In [ ]:
import autograd.numpy as np
from autograd import grad
import matplotlib.pyplot as plt


In [ ]:
# ===== ĐỔI HÀM Ở ĐÂY =====
def f(x):
    return (x[0] - 3)**2 + 2

x0 = np.array([0.0])


In [ ]:
def f(x):
    return np.sum((x - 3)**2) + 2


In [ ]:
grad_f = grad(lambda x: f(x))


In [ ]:
def backtracking_line_search(f, grad_f, x, p, alpha0=1.0, rho=0.5, c=1e-4):
    alpha = alpha0
    fx = f(x)
    g = grad_f(x)

    while f(x + alpha * p) > fx + c * alpha * np.dot(g, p):
        alpha *= rho
        if alpha < 1e-12:
            break
    return alpha


In [ ]:
def exact_line_search_quadratic(A, g):
    return (g @ g) / (g @ A @ g)


In [ ]:
def gradient_descent(
    f,
    grad_f,
    x0,
    lr=0.1,
    max_iter=100,
    tol=1e-8,
    use_backtracking=False,
    use_exact_line_search=False,
    A=None,          # chỉ dùng cho exact line search
    verbose=True
):
    x = x0.copy()
    xs = [x.copy()]
    fs = [f(x)]

    for k in range(max_iter):
        g = grad_f(x)

        if np.linalg.norm(g) < tol:
            if verbose:
                print(f"Hội tụ tại iter {k}, ||grad|| = {np.linalg.norm(g):.2e}")
            break

        p = -g

        # chọn step size
        if use_exact_line_search:
            if A is None:
                raise ValueError("Exact line search cần ma trận A")
            alpha = exact_line_search_quadratic(A, g)
        elif use_backtracking:
            alpha = backtracking_line_search(f, grad_f, x, p)
        else:
            alpha = lr

        x = x + alpha * p
        xs.append(x.copy())
        fs.append(f(x))

        if verbose:
            print(f"Iter {k+1:3d}: x = {x}, f(x) = {fs[-1]:.6f}, alpha = {alpha:.3e}")

    return np.array(xs), np.array(fs)


In [ ]:
def print_convergence_condition(P):
    eigvals = np.linalg.eigvals(P)
    L = np.max(np.real(eigvals))
    print("\n===== ĐIỀU KIỆN HỘI TỤ GD =====")
    print("lambda_max(P) =", L)
    print("Hội tụ nếu: 0 < lr < 2 / L =", 2/L)


In [ ]:
learning_rates = [0.01, 0.1, 0.5, 1.0]

histories = {}

for lr in learning_rates:
    print(f"\n===== GD với lr = {lr} =====")
    xs, fs = gradient_descent(
        f, grad_f, x0,
        lr=lr,
        max_iter=50,
        use_backtracking=False,
        verbose=False
    )
    histories[lr] = fs


In [ ]:
plt.figure(figsize=(8,5))
for lr, fs in histories.items():
    plt.semilogy(np.abs(fs - 2), label=f"lr={lr}")
plt.xlabel("Iteration")
plt.ylabel("|f(x) - f(x*)|")
plt.title("Gradient Descent – Bài 1.1")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
xs_bt, fs_bt = gradient_descent(
    f, grad_f, x0,
    lr=1.0,
    max_iter=50,
    use_backtracking=True,
    verbose=True
)


In [ ]:
A = np.array([[2.0]])  # Hessian của (x-3)^2

xs_exact, fs_exact = gradient_descent(
    f, grad_f, x0,
    max_iter=20,
    use_exact_line_search=True,
    A=A,
    verbose=True
)


In [ ]:
def print_convergence_condition(
    f,
    dim,
    sample_points=None,
    assume_quadratic=False
):
    """
    In điều kiện hội tụ Gradient Descent cho hàm f(x)

    Parameters
    ----------
    f : callable
        Hàm mục tiêu f(x), x là vector R^n
    dim : int
        Số chiều của x
    sample_points : list[np.ndarray], optional
        Danh sách điểm để ước lượng Hessian (nếu không phải quadratic)
    assume_quadratic : bool
        Nếu biết chắc f là quadratic → Hessian hằng
    """

    grad_f = grad(f)
    hess_f = hessian(f)

    print("\n==============================")
    print("ĐIỀU KIỆN HỘI TỤ GRADIENT DESCENT")
    print("==============================")

    if assume_quadratic:
        x0 = np.zeros(dim)
        H = hess_f(x0)
        eigvals = np.linalg.eigvals(H)
        L = np.max(np.real(eigvals))

        print("→ Hàm được giả sử là quadratic")
        print("Hessian =\n", H)
        print("Eigenvalues =", eigvals)
        print(f"L = lambda_max(H) = {L:.6f}")

    else:
        if sample_points is None:
            raise ValueError("Cần sample_points để ước lượng L")

        L_list = []
        for x in sample_points:
            H = hess_f(x)
            eigvals = np.linalg.eigvals(H)
            L_list.append(np.max(np.real(eigvals)))

        L = max(L_list)

        print("→ Hàm tổng quát (ước lượng Lipschitz constant)")
        print("Các giá trị L ước lượng:", L_list)
        print(f"L (max) ≈ {L:.6f}")

    print("\n👉 Điều kiện hội tụ Gradient Descent:")
    print(f"    0 < learning_rate < 2 / L = {2/L:.6f}")
    print("==============================\n")

    return L


In [ ]:
import autograd.numpy as np
from autograd import grad, hessian


In [ ]:
def f1(x):
    return (x[0] - 3)**2 + 2

L = print_convergence_condition(
    f=f1,
    dim=1,
    assume_quadratic=True
)


In [ ]:
def f2(x):
    return np.sum((x - 3)**2)

L = print_convergence_condition(
    f=f2,
    dim=5,
    assume_quadratic=True
)


In [ ]:
def rosenbrock(x):
    return (1-x[0])**2 + 100*(x[1] - x[0]**2)**2

sample_points = [
    np.array([-1.2, 1.0]),
    np.array([0.0, 0.0]),
    np.array([1.0, 1.0])
]

L = print_convergence_condition(
    f=rosenbrock,
    dim=2,
    sample_points=sample_points,
    assume_quadratic=False
)
